In [ ]:
from daft import serving

In [ ]:
BACKEND_CONFIGS = [
    {
        "name": "local",
        "config": {"type": "docker"},
    },
    {
        "name": "local-mp",
        "config": {"type": "multiprocessing"},
    },
    {
        "name": "aws_lambda",
        "config": {"type": "aws_lambda", "execution_role_arn": "arn:aws:iam::941892620273:role/jay-daft-serving-testrole-OKTODELETE"},
    },
]

In [ ]:
client = serving.ServingClient.from_configs(BACKEND_CONFIGS)

In [ ]:
client.list_backends()

In [ ]:
import numpy as np

def myfunc():
    return np.ones((10, 10)).mean().item()

In [ ]:
client.list_endpoints()

In [ ]:
client.deploy("docker_endpoint_myfunc", myfunc, backend="local", pip_dependencies=["numpy"])
client.deploy("docker_endpoint_myfunc", myfunc, backend="local-mp", pip_dependencies=["numpy"])
# client.deploy("lambda_endpoint_myfunc", myfunc, backend="aws_lambda", pip_dependencies=["numpy"])

In [ ]:
client.list_endpoints()

In [ ]:
import requests

In [ ]:
response = requests.get('http://localhost:50260')
response.text

In [ ]:
from daft.dataframe import DataFrame
from daft.serving.endpoint import HTTPEndpoint
from daft.expressions import col

source_df = DataFrame.from_pydict({"foo": [1, 2, 3]})
http_endpoint = HTTPEndpoint(source_df.schema())

df = DataFrame.from_http_endpoint(http_endpoint)
df = df.with_column("bar", col("foo") + 1)
df.write_http_endpoint(http_endpoint)

http_endpoint.deploy("my_df_endpoint", backend="local-mp", client=client)

In [ ]:
response = requests.get('http://localhost:50270?request=papa')
response.text